In [1]:

import quantit as qtt
import torch


Quantit offer several factory function to generate tensors,
and several possible conservation laws.
Currently The following consevations law can be used without writing any C++ extensions:

In [2]:
qtt.conserved.Z
qtt.conserved.ZC2
qtt.conserved.ZC3
qtt.conserved.ZC4
qtt.conserved.ZC5
qtt.conserved.ZC6
qtt.conserved.ZZ
qtt.conserved.ZZC2
qtt.conserved.ZZC3
qtt.conserved.ZZC4
qtt.conserved.ZZC5
qtt.conserved.ZZC6
qtt.conserved.C2
qtt.conserved.C3
qtt.conserved.C4
qtt.conserved.C5
qtt.conserved.C6
qtt.conserved.C2C2
qtt.conserved.C2C3
qtt.conserved.C2C4
qtt.conserved.C2C5
qtt.conserved.C2C6

<function quantit.quantit.conserved.PyCapsule.C2C6>


The Z stand for the natural numbers, It is good for conserved values can be mapped onto positive or negative values, zero included, such as z-axis angular momentum and particle number.
The C stands for cyclical, and the number following it is the length of the cycle.
For exemple C6 can take values from 0 to 5, and could be used to quantify the momentum of a particle on a 6 sites periodic system, such as an hexagonal plaquette.
qtt.conserved.ZZ means that our conservation law can be specified by 2 integers. For exemple, this particular one is pertinent when both particle and spin is conserved.

Those are so-called Abelian conservation laws, because they relate to abelian symmetry groups.
The key property of such conserved quantity is that the tensor product of two states with well defined conserved quantities also has a well defined conserved conserved quantities.
Such is not the case for every conserved quantities. For exemple, the total angular momentum doesn't, and isn't described by an Abelian group.

A btensor (block tensor) defined by QuantiT is built of blocks of the same rank has the overall tensor. In essence,
the full tensor is subdivided into sections along each of its dimensions. Each of the section has an associated conserved quantity and size.
The blocks are formed by the intersection of the section in each dimensions. Each block has a one conserved quantity on each of its dimensions.
Only the blocks that respect a selection rule are allowed to have non-zero elements.
A block respect the selection rule if the conserved quantities of each of its dimensions sums up to a predetermined value.
Forbiden blocks are not explicitly stored and permitted blocks of zeros need not be explicitly stored.

 exemple with a rank 2 tensor (matrix) of the inner structure
 of this type:


````
              S0,0 │ S0,1 │ S0,2 │ S0,3
             ╔═════╪══════╪══════╪═════╗
             ║     │      │      │     ║
         S1,0║(0,0)│ (0,1)│ (0,2)│(0,3)║
             ║     │      │      │     ║
            ─╫─────┼──────┼──────┼─────╢
             ║     │      │      │     ║
             ║     │      │      │     ║
         S1,1║(1,0)│ (1,1)│ (1,2)│(1,3)║
             ║     │      │      │     ║
             ║     │      │      │     ║
             ║     │      │      │     ║
            ─╫─────┼──────┼──────┼─────╢
             ║     │      │      │     ║
             ║     │      │      │     ║
         S1,2║(2,0)│ (2,1)│ (2,2)│(2,3)║
             ║     │      │      │     ║
             ║     │      │      │     ║
             ║     │      │      │     ║
             ╚═════╧══════╧══════╧═════╝
````
In the preceding exemple, the rows are separated in 4 sections, and the columns in 3 sections.
This make up to 12 blocks, that we label by section.
Let's consider that the conserved quantity is simply an integer under the addition,that the column sections [-2,-1,1], the row sections have the conserved quantity [1,2,3,-1] and the selection rule is 0. In that case, only
the blocks [(1,0),(0,1),(2,3)] can be non-zero.

to build a block tensor we must specify its shape:
Each dimension of the tensor is defined by a list of conserved quantites and sizes.
Let's consider a vector as an exemple:

In [3]:
Z = qtt.conserved.Z
V = qtt.sparse_zeros([[(2, Z(0)), (3, Z(1)), (1, Z(-1))]], Z(1))

 The tensor thus constructed is empty, it has 6 elements and the selection rules is Z(1).
 The first 2 element of the vector are associated with the conserved quantity Z(0), the next three Z(1) and the last one with Z(-1).
 Only the 3 middle elements can ever differ from zero.

In [4]:
FermionShape = qtt.btensor([[(1, Z(0)), (2, Z(1)), (1, Z(2))]], Z(0))

 Fermions shape describe the shape a ket half-spin fermions on a single site could take.
 This second exemple call a constructor of btensor instead of a factory function,
 and it build a vector in the local hilbert space of electrons.
 This constructor is equivalent to sparse_zero.
 This vector has 4 elements, one with 0 particles, 2 with one particles, and one with two particles.

 Complex conjugation of btensor also inverse the conserved quantities. Conjugating ``FermionShape`` tensor lends us the shape a bra should have.


In [5]:
ConjFermionShape = FermionShape.conj()
print([x for x in ConjFermionShape.sections_quantity(0)])

[[Z(0)], [Z(-1)], [Z(-2)]]


The shape of tensor can be composed into tensors of higher rank.
From the previous two vector shape, we can construct the shape of an operator in that Hilbert space.

In [6]:
Hilb_operatorShape = qtt.shape_from([FermionShape, ConjFermionShape])
print("Shape of electron's local hilber space, with particle number conservation:\n", Hilb_operatorShape)

Shape of electron's local hilber space, with particle number conservation:
 btensor rank 2
 selection rule [Z(0)]
 number of sections by dim [3, 3]
 sections sizes [1, 2, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-2)]]



There's a few different way to construct a tensor,
We will construct the fermion annihilation and creation operators to demonstrate 3. 
Block by block:

In [7]:
c_up = qtt.sparse_zeros_like(Hilb_operatorShape) #We create a new empty tensor with the same shape as Hilb_operatorShape
c_up.set_selection_rule_(Z(-1)) #we set the selection rule to -1: 
# only the blocks that reduce the number of particle by one can be non-zero
c_up.blocks[0, 1] = torch.Tensor([[1, 0]]) #we populate with the correct value the two relevent blocks
c_up.blocks[1, 2] = torch.Tensor([[0], [1]])
print("c_up", c_up)

c_up btensor rank 2
 selection rule [Z(-1)]
 number of sections by dim [3, 3]
 sections sizes [1, 2, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-2)]]
block at [0, 1]
  1  0
[ CPUFloatType(1,2) ]
block at [1, 2]
  0
 1
[ CPUFloatType(2,1) ]



by converting a full tensor into a block tensor:

In [8]:
c_dn = qtt.from_torch_tensor_like(
    Hilb_operatorShape.set_selection_rule(Z(-1)),
    torch.Tensor([[0, 0, 1, 0], [0, 0, 0, -1], [0, 0, 0, 0], [0, 0, 0, 0]]),
)
print("c_dn", c_dn)
# A downside of this second method is that null block that are allowed by the selection rule could be present.
# it's not a concern in this particular case.

c_dn btensor rank 2
 selection rule [Z(-1)]
 number of sections by dim [3, 3]
 sections sizes [1, 2, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-2)]]
block at [0, 1]
  0  1
[ CPUFloatType(1,2) ]
block at [1, 2]
 -1
 0
[ CPUFloatType(2,1) ]



by inserting lower rank tensors:

In [9]:
c_dagger_up = qtt.sparse_zeros_like(Hilb_operatorShape)
c_dagger_dn = qtt.sparse_zeros_like(Hilb_operatorShape)

c_dagger_up.basic_index_put_([-1,0],torch.Tensor([0,1,0,0]))
c_dagger_up.basic_index_put_([-1,2],torch.Tensor([0,0,0,1]))

c_dagger_dn.basic_index_put_([-1,0],torch.Tensor([0,0,1,0]))
c_dagger_dn.basic_index_put_([-1,1],torch.Tensor([0,0,0,-1]))
print(c_dagger_dn)
print(c_dagger_up)

btensor rank 2
 selection rule [Z(0)]
 number of sections by dim [3, 3]
 sections sizes [1, 2, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-2)]]

btensor rank 2
 selection rule [Z(0)]
 number of sections by dim [3, 3]
 sections sizes [1, 2, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-2)]]



The method *basic_index_put_* works by copying a lower rank tensor into a matching view into the target tensor. The view is specified using -1 for each dimension to be kept as is and an integer value other wise. The supplied values can be in the form of a torch tensor or a btensor of matching shape. When using a torch tensor, non-zero values that are disallowed in the target tensor are silently ignored. 

In the previous example, we insert values column by column into the creation operator. This method for filling in values in a tensor is used more extensively in the exemple MPO.py, to construct a rank 4 tensor from matrices.

Next we construct the creation operators (again) and number operators:

In [10]:
c_up_dag = c_up.conj().t_()# t_ permute that last two dimensions of a tensor, it transpose matrices
c_dn_dag = c_dn.conj().t_()# the underscore signifies that the operation is done "in-place" no new tensors created by the operation (conj already created a new one that t_ acts on )


print(c_up_dag)
print(c_dn_dag)

n_up = qtt.tensordot(c_up_dag,c_up,dims=([1],[0]))
n_dn = c_dn_dag.bmm(c_dn)

print("n_up ", n_up)
print("n_dn ",n_dn)

btensor rank 2
 selection rule [Z(1)]
 number of sections by dim [3, 3]
 sections sizes [1, 2, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-2)]]
block at [1, 0]
  1
 0
[ CPUFloatType(2,1) ]
block at [2, 1]
  0  1
[ CPUFloatType(1,2) ]

btensor rank 2
 selection rule [Z(1)]
 number of sections by dim [3, 3]
 sections sizes [1, 2, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-2)]]
block at [1, 0]
  0
 1
[ CPUFloatType(2,1) ]
block at [2, 1]
 -1  0
[ CPUFloatType(1,2) ]

n_up  btensor rank 2
 selection rule [Z(0)]
 number of sections by dim [3, 3]
 sections sizes [1, 2, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-2)]]
block at [1, 1]
  1  0
 0  0
[ CPUFloatType(2,2) ]
block at [2, 2]
  1
[ CPUFloatType(1,1) ]

n_dn  btensor rank 2
 selection rule [Z(0)]
 number of sections by dim [3, 3]
 sections sizes [1, 2, 1, 1, 2, 1]
 sections conserved quantity [[Z(0)], [Z(1)]

We created $n_\uparrow$ using the tensordot method, and $n_\downarrow$ using bmm. In this particular case, both operation are equivalent: they perform a matrix-matrix multiplication.
In general, tensordot is for performing an arbitrary contraction of two tensors while bmm perform a batched matrix multiplication.

Tensordot function by specifying 2 tensors and the dimension to contract with each other in a 2-tuple of lists named dims. 

bmm broadcast together the first N-2 dimension of two tensors, and performs a matrix multiplication on the last two.



### Factory functions

Quantity implment a set of factory function to automatically generate tensors with a simple values. All of those function come in two flavors, one taking a shape description and selection rule, and one taking a btensor to encode the shape. the factory function that fill the btensors with values will only fill the block that are allowed by the specified selection rule

Those factory function are the same set that are available for *torch.tensor*

In [11]:
qtt.sparse_zeros
qtt.sparse_zeros_like
qtt.zeros
qtt.zeros_like
qtt.ones
qtt.ones_like
qtt.empty
qtt.empty_like
qtt.rand
qtt.rand_like
qtt.full
qtt.full_like
qtt.randint
qtt.randint_like
qtt.randn
qtt.randn_like

<function quantit.quantit.PyCapsule.randn_like>

### Broadcasting and element by element operations

btensors support several broadcasting and element by element operations.

basic arithmetic (+ - * /) and comparison operator (> >= < <= == !=) can be done between two tensors, with broacasting for size 1 or absent dimensions.

The following function can be applied element by element to a tensor: 
- *sqrt* for the square root
- *abs* for the absolute value
- *sqrt_* for performing the square root computation in place
- *abs_* for performing the absolute value in place




### Linear algebra and Reshaping

Singular value decomposition and Eigenvalue decomposition are available in the *linalg* submodule of QuantiT.
They are present in two flavor: batched and Tensor networks.

The batched version are the equivalent of the methods implemented by torch, but take avantage of the block structure to reduce the numerical cost of the computation.

The tensor network version of the functions implicitly reshape the supplied tensor into a matrix before performing the routine and undoes it before returning the result.

Additionnaly the tensor network method can perform automatic truncation based on the values in the diagonal matrix produced by the decomposition.

In [12]:
X = qtt.rand_like(qtt.shape_from([c_dn,c_dn]))
# A rank 4 matrix.

U,d,V = qtt.linalg.svd(X,2,tol = 1e-3,min_size = 2 ,max_size = 100)

#d is the diagonnal of the matrix of singular values D.
print("U = ", U,'\n d=',d,'\n V= ',V)

X_reconstructed = qtt.tensordot((U*d),V.conj(),dims = ([U.dim()-1],[V.dim()-1]))
#We use broadcasting to do the U.D without building D explicitly.

T = (X_reconstructed-X) > 1e-5 #we're currently working in single precision, so we can't be too stringent.

print(qtt.any(T)) # we expect false.


U =  btensor rank 3
 selection rule [Z(-2)]
 number of sections by dim [3, 3, 3]
 sections sizes [1, 2, 1, 1, 2, 1, 1, 4, 1]
 sections conserved quantity [[Z(0)], [Z(1)], [Z(2)], [Z(0)], [Z(-1)], [Z(-2)], [Z(0)], [Z(-1)], [Z(-2)]]
block at [0, 0, 2]
 (1,.,.) = 
  0.4680
[ CPUFloatType(1,1,1) ]
block at [0, 1, 1]
 (1,.,.) = 
 -0.5225  0.6061  0.1906 -0.5687
 -0.4693 -0.7852  0.2391 -0.3256
[ CPUFloatType(1,2,4) ]
block at [0, 2, 0]
 (1,.,.) = 
 -1
[ CPUFloatType(1,1,1) ]
block at [1, 1, 2]
 (1,.,.) = 
  0.4932
  0.4615

(2,.,.) = 
  0.3621
  0.1759
[ CPUFloatType(2,2,1) ]
block at [1, 2, 1]
 (1,.,.) = 
 -0.4266 -0.0395 -0.9024  0.0473

(2,.,.) = 
 -0.5700  0.1205  0.3037  0.7539
[ CPUFloatType(2,1,4) ]
block at [2, 2, 2]
 (1,.,.) = 
  0.4034
[ CPUFloatType(1,1,1) ]
 
 d= btensor rank 1
 selection rule [Z(0)]
 number of sections by dim [3]
 sections sizes [1, 4, 1]
 sections conserved quantity [[Z(0)], [Z(0)], [Z(0)]]
block at [0]
  1.7459
[ CPUFloatType(1) ]
block at [1]
  2.1753
 0.780

The last 3 argument are optionnal and serve to specify the parameters of the truncation: tol is the permitted reconstruction error using (by default) the Frobenius norm. min_size and max_size control the size new index without regards to the truncation error.

The second argument specify the implicit matrix shape used: in this case the first 2 dimensions of the tensor are treated as the row of a matrix and rest are the columns.

The eigenvalue decomposition can also be done if the split result in a hermitian matrix with *eigh* and the same interface.

The reshaping that has taken place implicitly so far can be done explicitly using the function *qtt.reshape* and *qtt.reshape_as*. Lets do the tensor SVD using the batched SVD by explicitly reshaping the **X** tensor.

In [16]:
X_mat = qtt.reshape(X,[2])

Utmp,d,Vtmp = qtt.linalg.svd(X_mat)

U_shape = qtt.shape_from([X.shape_from([-1,-1,0,0]),Utmp.shape_from([0,-1])])
V_shape = qtt.shape_from([X.shape_from([0,0,-1,-1]).conj(),Vtmp.shape_from([0,-1])])

U = qtt.reshape_as(Utmp,U_shape)
V = qtt.reshape_as(Vtmp,V_shape)

qtt.linalg.truncate(U,d,V,tol=1e-3,min=1,max=100,pow=2)#truncate function for SVD


the *qtt.reshape* function takes a list of integers to specify a new shape of a lower rank than the input tensor. It simply join together all the dimension in the intervals of dimension between the value in the list.

In the previous exemple, the single *2* present means that dimensions [0,2[ are joined in a single dimension in the output, and [2,3] are joined in a single dimensions.

A rank three tensor could be specified by writing

*qtt.reshape(X,[1,2])* would join together the last 2 dimension of the tensor and leave the first 2 as they were. This method of reshaping can only reduce the rank of the tensor.

To do an arbitrary reshape (including ones that increase the rank), one must specify completly the desired shape in the form a another btensor with said shape.